In [ ]:
# Pkg.build("Plots") 
using Plots
using Colors

# Set the default plot size to something that fits a cell
default(size = (800, 300))

In [ ]:
infection_rate = 0.3
reinfection_rate = 0.02
death_probability = 0.01
recovery_time = 5

In [ ]:
@enum InfectionStatus uninfected infected dead recovered

mutable struct Cell
    status::InfectionStatus
    infection_time::Int8
end


function create_map(width::Int64=32, height::Int64=32)
    cells = Array{Cell}(undef, width, height)
    for i in 1:size(cells)[1]
        for j in 1:size(cells)[2]
            cells[i,j] = Cell(uninfected, 0)
        end
    end
    cells[2,3].status = infected
    return cells
end

# Map the cells to colors
function to_colors(cell::Cell)
    if cell.status == uninfected
        return RGB(0.0,0.8,0.0)
    end
    if cell.status == infected
        return RGB(0.8,0.0,0.0)
    end
    if cell.status == dead
        return RGB(0.1,0.1,0.1)
    end
    if cell.status == recovered
        return RGB(0.0,0.0,0.8)
    end
end

function interact!(new_cell::Cell, other_cell::Cell)
    if new_cell.status == uninfected && other_cell.status == infected
        if rand(1)[1] < infection_rate
            new_cell.status = infected
            new_cell.infection_time = 0
        end
    end
    if new_cell.status == recovered && other_cell.status == infected
        if rand(1)[1] < reinfection_rate
            new_cell.status = infected
            new_cell.infection_time = 0
        end
    end
end

function update!(new_cell::Cell)
    if new_cell.status == infected
        new_cell.infection_time += 1
        if new_cell.infection_time > recovery_time
            new_cell.status = recovered
        end
        if rand(1)[1] < death_probability
            new_cell.status = dead
        end
    end
end

function update(cells::Matrix{Cell})
    new_cells = deepcopy(cells)
    for i in 1:size(cells)[1]
        for j in 1:size(cells)[2]
            update!(new_cells[i,j])
        end
    end
    for i in 1:size(cells)[1]-1
        for j in 1:size(cells)[2]
            interact!(new_cells[i,j], cells[i+1,j])
            interact!(new_cells[i+1,j], cells[i,j])
        end
    end
    for i in 1:size(cells)[1]
        for j in 1:size(cells)[2]-1
            interact!(new_cells[i,j], cells[i,j+1])
            interact!(new_cells[i,j+1], cells[i,j])
        end
    end
    return new_cells
end


In [ ]:
cells = create_map()

anim = @animate for i ∈ 1:100
    if i > 1
        cells = update(cells)
    end
    plot!(to_colors.(cells), legend=false, border=:none)
end
gif(anim, "pandemic.gif", fps = 5)